In [ ]:
!pip install refractml==1.0.3
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 51kB 1.6MB/s eta 0:00:01
Processing /home/mosaic-ai/.cache/pip/wheels/ab/d0/0e/613976a1b51b5654859e2a82ade64329859bce431e280f2a39/shutils-0.1.0-cp38-none-any.whl
     |████████████████████████████████| 71kB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 143kB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 706kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 31.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 36.4MB/s eta 0:00:01
     |████████████████████████████████| 9.8MB 16.8MB/s eta 0:00:01     |███████▉                        | 2.4MB 16.8MB/s eta 0:00:01     |███████████████████████████▏    | 8.3MB 16.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 14.2MB/s eta 0:00:01
     |████████████████████████████████| 307kB 10.0MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 2.8MB/s eta 0:00:01     |██████████▏                     | 5.5MB 10.0MB/s eta 0:00:02
   

In [1]:
import pandas as pd
import numpy as np
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from refractml import *
from refractml.constants import MLModelFlavours

ModuleNotFoundError: No module named 'category_encoders'

In [2]:
cars = pd.read_csv("/data/car_new.csv")
cars

,buying_num,maintenance_num,doors_num,persons_num,lugboot_num,safety_num,class_num
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [3]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [4]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [5]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [6]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [7]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'buying_num': {'type': 'integer', 'format': 'int64'}, 'maintenance_num': {'type': 'integer', 'format': 'int64'}, 'doors_num': {'type': 'integer', 'format': 'int64'}, 'persons_num': {'type': 'integer', 'format': 'int64'}, 'lugboot_num': {'type': 'integer', 'format': 'int64'}, 'safety_num': {'type': 'integer', 'format': 'int64'}}}, 'example': [{'buying_num': 1, 'maintenance_num': 2, 'doors_num': 0, 'persons_num': 2, 'lugboot_num': 0, 'safety_num': 1}, {'buying_num': 3, 'maintenance_num': 2, 'doors_num': 2, 'persons_num': 2, 'lugboot_num': 2, 'safety_num': 2}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [2]}}


In [ ]:
register_model(model, 
               score, 
               name='Refractml_Sklearn_Python', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=False, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=True, 
               kyd_score = False
              )